<a target="_blank" href="https://colab.research.google.com/github.com/giordamaug/HELP/blob/v2.0/notebooks/tuning.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://www.kaggle.com/notebooks/welcome?src=https://github.com/giordamaug/HELP/blob/v2.0/notebooks/tuning.ipynb">
  <img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Colab"/>
</a>

### 1. Load optuna library

In [1]:
import numpy as np
import pandas as pd
import os, sys 
import optuna

### 2. Download the input files
For a chosen tissue (here `Kidney`), download from Zenodo [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.12597679.svg)](https://doi.org/10.5281/zenodo.12597679)
 the label file (here `Kidney_HELP.csv`, computed as in Example 1) and the attribute files (here BIO `Kidney_BIO.csv`, CCcfs `Kidney_CCcfs.csv`, and N2V `Kidney_EmbN2V_128.csv`).  

Skip this step if you already have these input files locally.

In [1]:
from HELPpy.utility.utils import pdread_csv_fromurl
tissue='Kidney'
X_bio = pdread_csv_fromurl(f"https://zenodo.org/records/11917458/files/{tissue}_BIO.csv", index_col=0)
X_n2v = pdread_csv_fromurl(f"https://zenodo.org/records/11917458/files/{tissue}_EmbN2V_128.csv", index_col=0)
X_cccfs = pdread_csv_fromurl(f"https://zenodo.org/records/11917458/files/{tissue}_CCcfs.csv", index_col=0)
Y = pdread_csv_fromurl(f"https://zenodo.org/records/11917458/files/{tissue}_HELP.csv", index_col=0)

### 3. Pre-processing and merge input files
We apply pre-processing only to the BIO and CCcfs attributes.
+ constant attributes removal, and
+ data scaling with z-score.  

We also replace `E` label (positive label) and `aE` and `sNE` labels as 0 label (negative label), to address binary `E vs NE` classification.

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
df_y = Y.replace({'aE': 0, 'sNE': 0, 'E': 1})
print(df_y.value_counts(normalize=False))
consts = X_bio.columns[X_bio.nunique() <= 1].values
print(f"Removing {len(consts)} constant features ...")
X_bio = X_bio.drop(consts, axis=1)
X_bio = pd.DataFrame(StandardScaler().fit_transform(X_bio), index=X_bio.index, columns=X_bio.columns)
X_cccfs = pd.DataFrame(StandardScaler().fit_transform(X_cccfs), index=X_cccfs.index, columns=X_cccfs.columns)
consts = X_cccfs.columns[X_cccfs.nunique() <= 1].values
print(f"Removing {len(consts)} constant features ...")
X_cccfs = X_cccfs.drop(consts, axis=1)
df_X = pd.merge(X_bio, X_n2v, left_index=True, right_index=True, how='outer')
df_X = pd.merge(df_X, X_cccfs, left_index=True, right_index=True, how='outer')
idxs = np.intersect1d(df_y.index.values, df_X.index.values)
df_X = df_X.loc[idxs]
df_y = df_y.loc[idxs]
dataset = pd.concat([df_X,df_y], axis=1)

label
0        16678
1         1253
Name: count, dtype: int64
Removing 0 constant features ...
Removing 3 constant features ...


### 4. Compare models (Pycaret)

Start tuning session of models to be compared.

In [6]:
from pycaret.classification import *
clf = setup(data = dataset, 
             target = 'label',
             numeric_imputation = 'mean',
             categorical_features = [], session_id = 444,
             fold_strategy = "stratifiedkfold", fold=5,
             verbose = True)

,Description,Value
0,Session id,444
1,Target,label
2,Target type,Binary
3,Original data shape,"(17236, 3457)"
4,Transformed data shape,"(17236, 3457)"
5,Transformed train set shape,"(12065, 3457)"
6,Transformed test set shape,"(5171, 3457)"
7,Numeric features,3456
8,Rows with missing values,78.1%
9,Preprocess,True


### 4.1. Customize the metrics

In [8]:
from sklearn.metrics import *
from imblearn.metrics import specificity_score
try: 
    [remove_metric(m) for m in ['Precision', 'F1', 'Kappa', 'Recall']]  # remove unused metrics
    add_metric('Sensitivity', 'Sensitivity', recall_score, greater_is_better = True)
    add_metric('Specificity', 'Specificity', specificity_score, greater_is_better = True)
    add_metric('Balanced Accuracy', 'BA', balanced_accuracy_score, greater_is_better = True)
except: 
    pass
get_metrics()

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x17c128f70>,accuracy,pred,{},True,True,False
auc,AUC,AUC,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(roc_auc_score, response_method=('d...",pred_proba,"{'average': 'weighted', 'multi_class': 'ovr'}",True,True,False
mcc,MCC,MCC,<function matthews_corrcoef at 0x17c129510>,"make_scorer(matthews_corrcoef, response_method...",pred,{},True,True,False
Sensitivity,Sensitivity,Sensitivity,<pycaret.internal.metrics.EncodedDecodedLabels...,"make_scorer(recall_score, response_method='pre...",pred,{},True,True,True
Specificity,Specificity,Specificity,<pycaret.internal.metrics.EncodedDecodedLabels...,"make_scorer(specificity_score, response_method...",pred,{},True,True,True
Balanced Accuracy,BA,BA,<pycaret.internal.metrics.EncodedDecodedLabels...,"make_scorer(balanced_accuracy_score, response_...",pred,{},True,True,True


#### 4.2. Tune models on E vs Ne classification for kidney case study

In [10]:
from icarlearn.ensemble.splitvotingens import *
savepath = "../pycaret"
tuned_models = []
tuners = []
hparams = {"n_voters" :[4, 6, 8, 10, 12, 14, 16, 18, 20]}
mymodels = [sveLGBM(),  sveXGB(), sveADA(), sveRF(), sveLDA(), sveET()]
mynames = ['sveLGBM',  'sveXGB', 'sveADA', 'sveRF', 'sveLDA', 'sveET']
for clfname,name in zip(mymodels,mynames):
    clf = create_model(clfname)
    tuned_model, tuner = tune_model(clf, optimize = 'BA', search_algorithm='grid', custom_grid=hparams, return_tuner=True)
    tuners += [tuner]
    save_model(tuned_model, os.path.join(savepath, f'saved_{name}_model'))
    
for clfname in ['lightgbm', 'xgboost', 'ada', 'rf', 'lda', 'lr', 'et', 'svm']:
    clf = create_model(clfname)
    tuned_model, tuner = tune_model(clf, optimize = 'BA', return_tuner=True)
    tuned_models += [tuned_model]
    tuners += [tuner]
    save_model(tuned_model, os.path.join(savepath, f'saved_{clfname}_model'))

results = compare_models(include=tuned_models, sort='BA')

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0,
               verbose=-1)
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step

,Accuracy,AUC,MCC,Sensitivity,Specificity,BA
Fold,,,,,,
0,0.8815,0.9581,0.5267,0.8960,0.8804,0.8882
1,0.8707,0.9525,0.4976,0.8736,0.8705,0.8720
2,0.8595,0.9342,0.4731,0.8621,0.8593,0.8607
3,0.8748,0.9528,0.5183,0.9023,0.8727,0.8875
4,0.8711,0.9616,0.5195,0.9195,0.8674,0.8934
Mean,0.8715,0.9518,0.5070,0.8907,0.8700,0.8804
Std,0.0071,0.0095,0.0195,0.0205,0.0069,0.0122


,Accuracy,AUC,MCC,Sensitivity,Specificity,BA
Fold,,,,,,
0,0.8599,0.9576,0.4967,0.9133,0.8558,0.8845
1,0.8537,0.9519,0.4879,0.9138,0.8490,0.8814
2,0.8313,0.9341,0.4363,0.8736,0.8280,0.8508
3,0.8550,0.9520,0.4952,0.9253,0.8495,0.8874
4,0.8504,0.9606,0.4987,0.9483,0.8428,0.8955
Mean,0.8501,0.9512,0.4830,0.9148,0.8450,0.8799
Std,0.0099,0.0092,0.0236,0.0242,0.0094,0.0153


Fitting 5 folds for each of 9 candidates, totalling 45 fits
Transformation Pipeline and Model Successfully Saved


,Accuracy,AUC,MCC,Sensitivity,Specificity,BA
Fold,,,,,,
0,0.8802,0.9589,0.5296,0.9075,0.8781,0.8928
1,0.8649,0.9494,0.4955,0.8908,0.8629,0.8768
2,0.8516,0.9345,0.4578,0.8563,0.8513,0.8538
3,0.8736,0.9519,0.5107,0.8908,0.8723,0.8815
4,0.8715,0.9621,0.5255,0.9310,0.8669,0.8990
Mean,0.8684,0.9514,0.5038,0.8953,0.8663,0.8808
Std,0.0097,0.0096,0.0260,0.0244,0.0091,0.0156


...

,Accuracy,AUC,MCC,Sensitivity,Specificity,BA
Fold,,,,,,
0,0.9399,0.9202,0.4858,0.4335,0.9790,0.7063
1,0.9412,0.9014,0.5035,0.4540,0.9790,0.7165
2,0.9407,0.8865,0.4914,0.4310,0.9803,0.7057
3,0.9424,0.9110,0.5132,0.4598,0.9799,0.7198
4,0.9399,0.9248,0.4964,0.4540,0.9777,0.7158
Mean,0.9408,0.9088,0.4981,0.4465,0.9792,0.7128
Std,0.0009,0.0137,0.0095,0.0118,0.0009,0.0058


,Accuracy,AUC,MCC,Sensitivity,Specificity,BA
Fold,,,,,,
0,0.8802,0.8411,0.3896,0.6185,0.9004,0.7595
1,0.8923,0.8209,0.4115,0.6092,0.9142,0.7617
2,0.8811,0.8347,0.3568,0.5517,0.9067,0.7292
3,0.8885,0.8318,0.4002,0.6034,0.9107,0.7571
4,0.8840,0.8111,0.3994,0.6207,0.9044,0.7626
Mean,0.8852,0.8279,0.3915,0.6007,0.9073,0.7540
Std,0.0046,0.0106,0.0187,0.0253,0.0048,0.0126


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Transformation Pipeline and Model Successfully Saved


,Model,Accuracy,AUC,MCC,Sensitivity,Specificity,BA,TT (Sec)
1,Extreme Gradient Boosting,0.7923,0.9348,0.4143,0.9287,0.7817,0.8552,3.1040
3,Random Forest Classifier,0.8102,0.9036,0.3892,0.8308,0.8086,0.8197,0.9120
6,Extra Trees Classifier,0.8261,0.8711,0.3763,0.7618,0.8311,0.7965,0.8620
4,Linear Discriminant Analysis,0.9455,0.9318,0.5918,0.6191,0.9709,0.7950,6.5100
5,Logistic Regression,0.8994,0.8424,0.4377,0.6272,0.9205,0.7739,1.5000
7,SVM - Linear Kernel,0.8852,0.8279,0.3915,0.6007,0.9073,0.7540,18.4220
2,Ada Boost Classifier,0.9437,0.9289,0.5341,0.4925,0.9787,0.7356,4.8420
0,Light Gradient Boosting Machine,0.9479,0.9406,0.5537,0.4741,0.9847,0.7294,2.0900


[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will b

#### 4.3. Compare tuned models

In [16]:
models = []
for clfname in ['sveLGBM',  'sveADA', 'sveRF', 'sveLDA', 'sveET', 
                'lightgbm', 'ada', 'rf', 'lda', 'lr', 'et', 'svm']:
    models += [load_model(os.path.join(savepath, f'saved_{clfname}_model'))]    
results = compare_models(include=models, sort='BA')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:45:21
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,sveADA


,Model,Accuracy,AUC,MCC,Sensitivity,Specificity,BA,TT (Sec)
0,sveLGBM,0.8501,0.9512,0.4830,0.9148,0.8450,0.8799,15.9400


Processing:   0%|          | 0/53 [00:00<?, ?it/s]

#### 4.4. Show list of ranked classifiers

In [15]:
df = pull()
df.to_csv("../results/pycaret_best_tuned_classifier_metrics.csv", index=True)
print(df.to_latex())

\begin{tabular}{llrrrrrrr}
\toprule
 & Model & Accuracy & AUC & MCC & Sensitivity & Specificity & BA & TT (Sec) \\
\midrule
0 & sveLGBM & 0.850100 & 0.951200 & 0.483000 & 0.914800 & 0.845000 & 0.879900 & 14.608000 \\
1 & sveADA & 0.856900 & 0.945400 & 0.487400 & 0.901100 & 0.853500 & 0.877300 & 13.146000 \\
4 & sveET & 0.866600 & 0.936400 & 0.480800 & 0.852700 & 0.867600 & 0.860200 & 3.588000 \\
2 & sveRF & 0.883200 & 0.938600 & 0.502100 & 0.832000 & 0.887200 & 0.859600 & 3.008000 \\
7 & Random Forest Classifier & 0.810200 & 0.903600 & 0.389200 & 0.830800 & 0.808600 & 0.819700 & 0.916000 \\
10 & Extra Trees Classifier & 0.826100 & 0.871100 & 0.376300 & 0.761800 & 0.831100 & 0.796500 & 0.758000 \\
8 & Linear Discriminant Analysis & 0.945500 & 0.931800 & 0.591800 & 0.619100 & 0.970900 & 0.795000 & 6.512000 \\
3 & sveLDA & 0.740800 & 0.856100 & 0.320500 & 0.837800 & 0.733300 & 0.785500 & 5.074000 \\
9 & Logistic Regression & 0.899400 & 0.842400 & 0.437700 & 0.627200 & 0.920500 & 0.773900 

### 5. Show optimal hyper-paramenters of sveLGB for E vs NE classification in kidney case sturdy

In [3]:
import pandas as pd
print(pd.read_csv("../results/svelgbm_Kidney_hypersearch_ba_cv.csv").replace({'':'','':'','':'','':''}).sort_values('value', ascending=False).to_latex()) 

\begin{tabular}{lrrrllllrrrl}
\toprule
 & Unnamed: 0 & number & value & datetime_start & datetime_complete & duration & params_boosting_type & params_learning_rate & params_n_estimators & params_n_voters & state \\
\midrule
37 & 37 & 37 & 0.893151 & 2024-06-05 11:48:54.638670 & 2024-06-05 11:50:51.129284 & 0 days 00:01:56.490614 & gbdt & 0.094505 & 200 & 13 & COMPLETE \\
15 & 15 & 15 & 0.891459 & 2024-06-05 11:16:36.759285 & 2024-06-05 11:17:48.725832 & 0 days 00:01:11.966547 & gbdt & 0.098300 & 140 & 10 & COMPLETE \\
44 & 44 & 44 & 0.890954 & 2024-06-05 12:01:09.467990 & 2024-06-05 12:03:01.279431 & 0 days 00:01:51.811441 & gbdt & 0.076452 & 200 & 12 & COMPLETE \\
43 & 43 & 43 & 0.890826 & 2024-06-05 11:59:18.529245 & 2024-06-05 12:01:09.439684 & 0 days 00:01:50.910439 & gbdt & 0.075168 & 200 & 12 & COMPLETE \\
41 & 41 & 41 & 0.890602 & 2024-06-05 11:55:19.637376 & 2024-06-05 11:57:14.427060 & 0 days 00:01:54.789684 & gbdt & 0.078591 & 200 & 13 & COMPLETE \\
33 & 33 & 33 & 0.890241 & 